In [ ]:
# Step 1: Install required packages
!pip install feedparser beautifulsoup4 openai

In [100]:
# Step 2: Download and parse the RSS feed
import feedparser
def get_feed(url):
    return feedparser.parse(url)

In [101]:
# Step 3: Extract HTML from the description
from bs4 import BeautifulSoup

def extract_text(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text()

In [102]:
import openai
openai.api_key = "insert-openAI-API-here"


# Step 4: Summarize using GPT-3
def summarize_text(text):
    response = openai.Completion.create(
        engine="text-davinci-003", 
        prompt=f"remove all unnecesary whitespace and formatting, then provide a short summary of less than 30 words of the following text: {text}",
        temperature=0.7,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

# Step 5: Sentiment classification
def sentiment_classification(text):
    response = openai.Completion.create(
        engine="text-davinci-003", 
        prompt=f"please classify the sentiment of this breaking financial news article on a range of -1 to 1 with 0.1 granularity, where -1 is maximum bearishness and 1 is maximum bullishness in relation to equities: {text}",
        temperature=0.7,
        max_tokens=50,
        top_p=1,
        frequency_penalty=0.5,
        presence_penalty=0,
    )
    return response.choices[0].text.strip()

In [103]:
# Step 6: Store the summarized information in a SQLite database
import sqlite3

def create_db():
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE news (title TEXT, summary TEXT, sentiment TEXT)''')
    return conn, cursor

def insert_news(cursor, title, summary, sentiment):
    cursor.execute("INSERT INTO news (title, summary, sentiment) VALUES (?, ?, ?)", (title, summary, sentiment))


In [110]:
# Step 7: Continuously monitor the RSS feed for updates
import time

def monitor_feed(url, conn, cursor, interval=60):
    seen_titles = set()
    while True:
        feed = get_feed(url)
        # print(feed.entries[0])
        for entry in feed.entries:
            title = entry.title
            if title not in seen_titles:
                if entry.category == "News" or entry.category =="Central Banks" or entry.category =="Technical Analysis":
                    seen_titles.add(title)
                    category = extract_text(entry.category)
                    published = extract_text(entry.published)
                    description = extract_text(entry.description)
                    summary = summarize_text(description[:4000])
                    sentiment = sentiment_classification(description[:4000])  
                    insert_news(cursor, title, summary, sentiment)  
                    print(f"{published} >>> {category} >>> {title} >>> Summary: {summary} >>> Sentiment: {sentiment}")
        time.sleep(interval)

In [111]:
'''
# This code will download the RSS feed and process new entries every 60 seconds. 
Adjust the interval parameter in monitor_feed() to control how often the feed is checked.
'''
if __name__ == "__main__":
    url = "https://www.forexlive.com/feed"
    conn, cursor = create_db()
    monitor_feed(url, conn, cursor)

Tue, 18 Apr 2023 01:15:16 GMT >>> Central Banks >>> PBOC sets USD/ CNY reference rate for today at 6.8814 (vs. estimate at 6.8828) >>> Summary: PBOC sets reference rate for the onshore yuan, allowing +/-2% of USD/CNY, no restrictions for USD/CNH. 38bn yuan injected in OMOs. >>> Sentiment: 0.0
Tue, 18 Apr 2023 00:52:39 GMT >>> News >>> "ChatGPT outperforms traditional sentiment analysis methods" to forecast stock prices >>> Summary: ChatGPT language models can predict stock market returns with sentiment analysis of news headlines, outperforming traditional sentiment analysis methods. >>> Sentiment: 0.7
Tue, 18 Apr 2023 00:41:45 GMT >>> Central Banks >>> BOJ is considering a 2025 CPI forecast that is still under target >>> Summary: BOJ is considering a projection to raise consumer prices in 2025; new Governor Ueda will be at first policy meeting. BOJ's communication policies could lead to market volatility. >>> Sentiment: 0.4
Tue, 18 Apr 2023 00:23:41 GMT >>> Central Banks >>> PBOC is ex

KeyboardInterrupt: ignored